# Introduction <a name="introduction"></a>

This notebook demonstrates how to create and manage an external model. It covers the setup of the model, various deployment methods, and effective tracking throughout its lifecycle. 



For detailed documentation on the `watsonx.governance` Factsheets Python client, please visit the [official documentation](https://s3.us.cloud-object-storage.appdomain.cloud/aifactsheets-client/index.html).

**Required Services:**
- `watsonx.governance`

**Required Packages:**
- **IBM Facts Client Python SDK (>=1.0.47)**




<a name="setupcloud"></a>

In [3]:
!pip install ibm-aigov-facts-client
!pip install -U python-dotenv --quiet

In [17]:
import os
import warnings
import wget
import shutil
import json
from dotenv import load_dotenv
from ibm_aigov_facts_client import AIGovFactsClient,CloudPakforDataConfig,DeploymentDetails,TrainingDataReference,ExternalModelSchemas,ModelDetails
from IPython.display import display, Markdown


shutil.rmtree('./mlruns', ignore_errors=True)
load_dotenv()

True

- This sample will use IBM Cloud by default. If you prefer to IBM watsonx.goverance software, set `use_software=True`
- Flag `run_cleanup_at_end` offers option to delete created assets at the end of the notebook.The notebook will show URL to UI for model and model use case at certain cells. By dafault we set it to `run_cleanup_at_end=False` so you can access UI and see the changes. If you decide to cleanup assets at the end, set `run_cleanup_at_end=True` and remember cells showing links to UI will `NOT` work in that case.

In [6]:
use_software=True
run_cleanup_at_end=True

- `Experiment`  be customized.


In [8]:
experiment_name="External Model"

---
## Authentication Setup<a name="setup"></a>

### IBM Cloud  <a name="IBM-Cloud"></a>

Your Cloud API key can be generated by going to the Users section of the Cloud console. From that page, go to **Manage->Access(IAM)->API keys-> Create**. Give your key a name and click Create, then copy the created key and use as API_KEY.

NOTE: You can also get OpenScale API_KEY using IBM CLOUD CLI.

How to install IBM Cloud (bluemix) console: instruction

How to get api key using console:

```
bx login --sso
bx iam api-key-create 'my_key'
```
- Get relevant space id from UI `(Deployments -> Spaces-> open space -> Manage -Space GUID)`

In [9]:
if not use_software:
    API_KEY=os.getenv("CLOUD_API_KEY", "<if you can't use .env you can provide your value here>")

[back to top](#introduction)
### IBM watsonx.goverance software<a name="Watsonx.Gov-Platform"></a>

- Service url is the watsonx.goverance software platform host URL. For skytap environment, it would be the internal nginx URL.
- You can either use user `password` or platform `apikey` to authenticate

In [10]:
if use_software:
    creds=CloudPakforDataConfig(service_url=os.getenv("CPD_SERVICE_URL", "<if you can't use .env you can provide your value here>"),
        username=os.getenv("CPD_USERNAME", "<if you can't use .env you can provide your value here>"),
        password=os.getenv("CPD_PASSWORD", "<if you can't use .env you can provide your value here>"))


[back to top](#introduction)
## Client Initialization
- Container type would be either `space` or `project`. To use get/set environment utilities, model asset should be stored in Space.
- If running this notebook multiple times with same experiment name or anytime face error saying `Experiment with same name already exists`, use `set_as_current_experiment=True` when initiating client

[back to top](#introduction)



In [11]:
if use_software:
   facts_client = AIGovFactsClient(cloud_pak_for_data_configs=creds,experiment_name=experiment_name,set_as_current_experiment=True,external_model=True)
else: 
   facts_client = AIGovFactsClient(api_key=API_KEY,experiment_name=experiment_name,set_as_current_experiment=True,external_model=True)

2024/08/05 10:44:10 INFO : Experiment External Model does not exist, creating new experiment
2024/08/05 10:44:10 INFO : Experiment successfully created with ID 142482313712788761 and name External Model
2024/08/05 10:44:10 INFO : Autolog enabled Successfully


In [12]:
facts_client.version

'1.0.79'

---
## Create Model Properties <a name="createprop"></a>

- To demonstrate custom facts, we need to create a placeholder model on which we can show the APIs.
- The model itself does not matter for custom fact demonstration, so we will just create a sample one quickly.
- In a real end-to-end scenario, the model would be created based on real SageMaker/Azure/Custom information and use real names and IDs from these environments.

For more details, see the [documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/factsheet-external.html).


### Essential Information Required for External Model Integration


- Input and Output schema details
- Training data reference information
- Model deployment details from the platform where the model is stored


### <span style="color: #FFD700;">Input and Output Schema Details</span>


In [15]:
!rm external_model_facts_payload.json
wget.download("https://raw.githubusercontent.com/IBM/ai-governance-factsheet-samples/main/Assets/data/external_model_facts_payload.json")

rm: external_model_facts_payload.json: No such file or directory


'external_model_facts_payload.json'

In [18]:
with open("external_model_facts_payload.json", 'r') as f:
    payload = json.load(f)

In [19]:
external_schemas=ExternalModelSchemas(input=payload["schemas"]["input"],output=payload["schemas"]["output"])

### <span style="color: #FFD700;">Training Data Reference Details</span> <a name="htd"></a>

The **Training Data Reference** is crucial for specifying the details and configuration required to access and utilize training data. This includes defining parameters such as the `schema, unique identifiers, type of data, and connectivity details`. Properly configuring these aspects ensures accurate and effective integration of training data, which is essential for building and deploying external models.


In [20]:
# Derive training data reference schema from model training input schema

training_data_schema={}

input_schema= payload["schemas"]["input"][0]
training_data_schema["schema"]=input_schema

In [21]:
TRAINING_DATA_ASSSET_NAME = "german_credit_data_biased_training.csv"
TYPE = "url"

External_data_url = "https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv"

- The **TrainingDataReference()** function is employed for creating the train_data_ref, which includes parameters like training schema, ID, type, and connectivity details.
- This formal approach ensures the accurate configuration and connection of training data for seamless external model creation.

In [22]:
tdataref = TrainingDataReference(schema=training_data_schema, id=TRAINING_DATA_ASSSET_NAME, type=TYPE, connection={"url":External_data_url})

### <span style="color: #FFD700;">Model Deployment Details and Model Details</span>

- **Model Deployment Details**: Includes parameters such as the identifier, name, deployment type, scoring endpoint, and description.

- **Model Details**: Covers attributes like the model type, input type, algorithm, label column, label type, and prediction type.


In [23]:
endpoint_name="sagemaker-scikit-learn-2023-08-11-09-00-35-455"
endpoint_url = "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations"
deployment_name = "Loan Application Model Deployment"

#### Model Deployment Details

In [24]:
deployment=DeploymentDetails(identifier=endpoint_name,name=deployment_name,deployment_type="online",scoring_endpoint=endpoint_url,description="Model deployed in AWS SageMaker to predict the loan application approval")



#### Model Details

In [25]:
model_details = ModelDetails(model_type="scikit-learn_1.1",input_type="Structured", algorithm="GradientBoost Algorithm",label_column="Risk", label_type="String",prediction_type="Binary")

## Save External Model to Inventory <a name="save"></a>

- This function facilitates the storage of external models within the inventory. It requires essential parameters including `endpoint_name`, `deployment`, and `model_details`.


- If the `Inventory_id` is not specified, the function will use the default Inventory ID from the platform's available assets. Nevertheless, all other specified parameters must be provided.

##### ⚠️ Warning: Use of `catalog_id`

- The `catalog_id` parameter can specify either an inventory ID or a catalog ID. Inventories, optimized for watsonx.governance, are recommended.
- Catalogs are still supported but will be deprecated over time. As a best practice, use inventories for storing use cases and external models.

For further details, please consult [the documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/factsheet-external.html).


In [26]:
model_name = "Loan Application Model"

In [28]:
external_model=facts_client.external_model_facts.save_external_model_asset(model_name,model_name,model_details=model_details,training_data_reference=tdataref,schemas=external_schemas,deployment_details=deployment,description="Model developed in AWS SageMaker to predict the loan application approval")

2024/08/05 10:45:27 INFO : External model asset saved successfully under asset_id 6fabcd7b-7093-44b0-a99b-24035e360d87 and catalog d8d10c03-1ba0-49ff-ac58-f5df932ae29b
2024/08/05 10:45:34 INFO : Current model information: {'asset_id': '6fabcd7b-7093-44b0-a99b-24035e360d87', 'container_type': 'catalog', 'container_id': 'd8d10c03-1ba0-49ff-ac58-f5df932ae29b', 'facts_type': 'modelfacts_user'}


In [29]:
external_model.get_info(True)

{'name': 'Loan Application Model',
 'description': 'Model developed in AWS SageMaker to predict the loan application approval',
 'asset_type': 'model_stub',
 'url': 'https://cpd-aigov.apps.fs-hotfix-test1.cp.fyre.ibm.com/data/catalogs/d8d10c03-1ba0-49ff-ac58-f5df932ae29b/asset/6fabcd7b-7093-44b0-a99b-24035e360d87/asset-preview?context=cpdaas',
 'asset_id': '6fabcd7b-7093-44b0-a99b-24035e360d87',
 'container_type': 'catalog',
 'container_id': 'd8d10c03-1ba0-49ff-ac58-f5df932ae29b',
 'facts_type': 'modelfacts_user'}

---

## Deployment Methods

These deployment methods are specifically tailored for external models. They detail the processes and strategies for effectively deploying models into environments, ensuring they meet operational requirements and integrate smoothly with existing systems.


### Retrieve Deployments for External Models

This function allows you to access and view all existing deployments associated with the external models. It provides detailed information on each deployment, including their status and configuration.


In [30]:
deployments = external_model.get_deployments()
for deployment in deployments:
    print(deployment)

2024/08/05 10:45:39 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model",
  "model_asset_id": "6fabcd7b-7093-44b0-a99b-24035e360d87",
  "model_container_type": "catalog",
  "model_container_id": "d8d10c03-1ba0-49ff-ac58-f5df932ae29b"
}


### Add New Deployment to External Model

This function allows you to add a deployment to the external model within the system. 

Note that this action will not affect external providers such as AWS or Azure. You can add multiple deployments simultaneously.




In [31]:

deployment_new = [{"id":"Loan Application Deploy","name":"Loan Application Model Deployment with modifications","type":"online","scoring_endpoint":endpoint_url,"description":"Model deployed in AWS SageMaker to predict the loan application approval"}]
deployments_list = external_model.add_deployments(deployment_new)

deployments_list[0]

2024/08/05 10:45:45 INFO : Deployment added successfully to an external model


{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": "false",
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model",
  "model_asset_id": "6fabcd7b-7093-44b0-a99b-24035e360d87",
  "model_container_type": "catalog",
  "model_container_id": "d8d10c03-1ba0-49ff-ac58-f5df932ae29b"
}

In [32]:

deployment_tuning = [{"id":"Loan Application Deploy With Tuning","name":"Loan Application Model Deployment with Tuning","type":"online","scoring_endpoint":endpoint_url,"description":"Model deployed in AWS SageMaker to predict the loan application approval"}]
deployments_list = external_model.add_deployments(deployment_tuning)

2024/08/05 10:45:50 INFO : Deployment added successfully to an external model


In [33]:
deployments_all = external_model.get_deployments()
for deployment in deployments_all:
    print(deployment)

2024/08/05 10:45:54 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model",
  "model_asset_id": "6fabcd7b-7093-44b0-a99b-24035e360d87",
  "model_container_type": "catalog",
  "model_container_id": "d8d10c03-1ba0-49ff-ac58-f5df932ae29b"
}
{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict t

### Methods for  External Model Deployments

Explore the various methods for managing and configuring external model deployments.


In [34]:
print("Deployment name is : {}".format(deployment.get_name()))
print("Deployment ID is : {}".format(deployment.get_id()))
print("Deployment description is : {}".format(deployment.get_description()))
print("Deployment type is : {}".format(deployment.get_type()))
print("Deployment scoring endpoint is : {}".format(deployment.get_scoring_endpoint()))
print("Deployment model id is : {}".format(deployment.get_model_id()))
print("Deployment model name is : {}".format(deployment.get_model_name()))

Deployment name is : Loan Application Model Deployment with Tuning
Deployment ID is : 79fd918459f44f9eccb0b7712c8700a1
Deployment description is : Model deployed in AWS SageMaker to predict the loan application approval
Deployment type is : online
Deployment scoring endpoint is : https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations
Deployment model id is : 6fabcd7b-7093-44b0-a99b-24035e360d87
Deployment model name is : Loan Application Model


### Update Deployment Attributes: Description, Scoring Endpoint, and Type


In [35]:
deployment.set_description(value="update - Model deployed in AWS SageMaker to predict the loan application approval")
deployment.set_scoring_endpoint(value="https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations-update")
deployment.set_type("update-online")

2024/08/05 10:46:12 INFO : Description is updated for deployment successfully
2024/08/05 10:46:16 INFO : Scoring endpoint is updated for deployment successfully
2024/08/05 10:46:21 INFO : Type is updated for deployment under model_stub successfully
2024/08/05 10:46:23 INFO : Type is updated for deployment under modelfacts_system successfully


In [36]:
print("Deployment description is : {}".format(deployment.get_description()))
print("Deployment type is : {}".format(deployment.get_type()))
print("Deployment scoring endpoint is : {}".format(deployment.get_scoring_endpoint()))

Deployment description is : update - Model deployed in AWS SageMaker to predict the loan application approval
Deployment type is : update-online
Deployment scoring endpoint is : https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations-update


### Delete Deployments

Users can delete multiple deployments simultaneously. 

This action performs a soft delete, updating the **`is_deleted`** attribute from `false` to `true`.


In [37]:
deployment_ids=[deployment.get_id()]
external_model.delete_deployments(deployment_ids)

2024/08/05 10:46:34 INFO : Deployment 79fd918459f44f9eccb0b7712c8700a1 deleted successfully


In [38]:
deployments_all = external_model.get_deployments()
for deployment in deployments_all:
    print(deployment)

2024/08/05 10:46:36 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model",
  "model_asset_id": "6fabcd7b-7093-44b0-a99b-24035e360d87",
  "model_container_type": "catalog",
  "model_container_id": "d8d10c03-1ba0-49ff-ac58-f5df932ae29b"
}
{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict t



---
## Creation of New AI Use Case <a href="#add_mut"></a>

An **AI Use Case** tracks model asset lifecycles across environments like development, pre-production, and production. 

**Note:** The term "AI Use Case" has replaced "Model Use Case" to reflect a broader range of AI assets. While some APIs may still use the old terminology, it will be phased out.

- If `ai_usecase_id` is not provided, the default inventory_id is used (requires `EDITOR` access).

- Retrieve the AI Use Case ID from the URL in inventory or by using `get_ai_usecase()`.

- For Cloud Pak for Data, ensure OpenPages integration is disabled (create inventory permission needed).


In [39]:
ai_usecase_inventory_id = os.getenv("INVENTORY_ID", "<if you can't use .env you can provide your inventory id here>")
ai_usecase_name="Automatic Loan classification"
ai_usecase_desc="AI usecase for Loan classification"

##### ⚠️ Warning: Use of `catalog_id`

- The `catalog_id` parameter can specify either an inventory ID or a catalog ID. Inventories, optimized for watsonx.governance, are recommended.


- Catalogs are still supported but will be deprecated over time. As a best practice, use inventories for storing use cases and external models.


In [40]:
ai_usecase = facts_client.assets.create_ai_usecase(catalog_id=ai_usecase_inventory_id,name=ai_usecase_name,description=ai_usecase_desc)
ai_usecase.get_info(True)

2024/08/05 10:46:56 INFO : AI usecase created successfully


{'name': 'Autmatic Iris classification',
 'description': 'AI usecase for iris classification',
 'asset_type': 'model_entry',
 'url': 'https://cpd-aigov.apps.fs-hotfix-test1.cp.fyre.ibm.com/data/catalogs/9df1cb81-e4d1-4b8b-9825-460f84b55dea/asset/b6713b82-5730-4cff-8de5-050c13b0b80d?context=cpdaas',
 'model_usecase_id': 'b6713b82-5730-4cff-8de5-050c13b0b80d',
 'container_type': 'catalog',
 'catalog_id': '9df1cb81-e4d1-4b8b-9825-460f84b55dea',
 'facts_type': 'model_entry_user'}

---

## Create an Approach <a href="#createapproach"></a>

- Track multiple models and prompts under a single use case by grouping them into different approaches.
- Create multiple approaches for various classification algorithms to facilitate comparison and integration.
- Use approaches to manage different models that need to be combined for a specific use case.


In [41]:
external_approach = ai_usecase.create_approach(name="externalapproach",description="approach for external AWS model")
external_approach.get_info()

2024/08/05 10:47:02 INFO : Approach created successfully


{'approach_id': '57d52037-fd74-4c65-9287-35f104964072',
 'approach_name': 'externalapproach',
 'approach_desc': 'approach for external AWS model',
 'model_asset_id': 'b6713b82-5730-4cff-8de5-050c13b0b80d',
 'model_container_type': 'catalog',
 'model_container_id': '9df1cb81-e4d1-4b8b-9825-460f84b55dea'}

---
## Track a External Model Under an AI Use Case <a name="add_mu"></a>


- **AI Use Cases** are designed to monitor the lifecycle of model assets across various stages, including development, pre-production, and production.
- To effectively integrate a model into an AI use case, three critical elements must be addressed: the **model**, the **AI use case**, and the **approach**.
- Link an existing AI use case by using the following method: `model.track(usecase=<ai_usecase>, approach=<approach1>, version_number="<Version Number>")`.

- **Version Numbers** are categorized as follows:
    - **Major Version:** Indicates significant changes, represented as `1.0.0`.
    - **Minor Version:** Reflects incremental improvements, represented as `0.1.0`.
    - **Patch Version:** Denotes minor fixes or updates, represented as `0.0.1`.
    - **Custom Version:** Allows for tailored versioning according to specific user needs.

- Ensure that the `ai_usecase`, `approach`, and `version_number` parameters are provided as mandatory.


In [42]:
external_model.track(usecase=ai_usecase,approach=external_approach,version_number="major",version_comment="external model major version")

-----------------------------------------------------------------------------------------------------------------------------
                                                   Tracking Process Started                                                  
-----------------------------------------------------------------------------------------------------------------------------
2024/08/05 10:47:09 INFO : Assigned externalapproach to Autmatic Iris classification for tracking.
2024/08/05 10:47:11 INFO : Initiate linking model to existing AI usecase b6713b82-5730-4cff-8de5-050c13b0b80d
2024/08/05 10:47:15 INFO : Successfully finished linking Model 6fabcd7b-7093-44b0-a99b-24035e360d87 to AI usecase


{'model_entry_catalog_id': '9df1cb81-e4d1-4b8b-9825-460f84b55dea',
 'model_entry_id': 'b6713b82-5730-4cff-8de5-050c13b0b80d',
 'model_entry_name': 'Autmatic Iris classification',
 'model_entry_status': 'Draft',
 'version_number': '1.0.0',
 'version_comment': 'external model major version',
 'approach_name': 'externalapproach'}

---

## External Model Environment Type <a name="existing_env_type"></a>

This section details the various environment types available for external models. It also discusses the process for promoting external model across different environment stages.


In [43]:
external_model.get_environment_type()

{'classification': 'TEST',
 'reason': 'The space type is development and deployment_details are available which is considered as TEST environment and asset shows under TEST stage'}

#### Model Promotion Across Pillars

Models move through different pillars based on their environment and evaluation status. Here’s how the transition works:

- **Develop**: The initial stage where models are created and developed.

- **Test**: Once promoted to a space, the model appears in the Test pillar.

- **Validate**: If a model's deployment is evaluated in Watson OpenScale and tagged as pre-production, it will be displayed in the Validate pillar. Additionally, models promoted to spaces with the `AIGovernance: Pre-production` tag will also be shown in the Validate pillar.

- **Operate**: Models tagged as production or those evaluated and tagged as production in Watson OpenScale will be displayed in the Operate pillar. Models promoted to spaces with the `AIGovernance: Production` tag will be shown in the Operate pillar.

#### -> Promote External Model from Test to Validate

In [44]:
external_model.set_environment_type(from_container="test",to_container="validate")

2024/08/05 10:47:26 INFO : Asset successfully moved from test to validate environment


#### ->Promote External Model from Validate to Operate

In [ ]:
external_model.set_environment_type(from_container="validate",to_container="operate")

#### -> Promote External Model from Operate to Validate

In [46]:
external_model.set_environment_type(from_container="operate",to_container="validate")

2024/08/05 10:47:43 INFO : Asset successfully moved from operate to validate environment


---
## Deployment Environment Type <a name="5"></a>

This section outlines the scenarios for external model deployment environments and promotions:

- **Models Without Deployment**: These models are located in the Develop pillar, where they are in the initial development phase.
  
- **Models With Deployment**:
  - **`Multiple Deployments`**: Moving a single deployment results in the creation of a clone of the entire model in the target pillar, with the deployment placed there.
  
  - **`Single Deployment`**: Moving the deployment causes the entire model to be migrated to the target pillar.

- **Complete Model Migration**: When a model is moved to a target pillar, it includes all associated deployments, whether there is one or multiple. Once a model is in a target pillar, it cannot be moved further.

- **OpenScale Evaluations**: Deployments evaluated by OpenScale will remain in their current pillar and cannot be relocated to another pillar.

**Notes**:
- Models with deployments cannot be moved from the Test, Validate, or Operate Pillars back to the Develop Pillar.


In [47]:
deployments_all = external_model.get_deployments()
for deployment in deployments_all:
    print(deployment)

2024/08/05 10:47:45 INFO : Deployments retrieved successfully
{
  "id": "99ba2b29cf98ae401fa4443fd9618fbd",
  "name": "Loan Application Model Deployment",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict the loan application approval",
  "model_name": "Loan Application Model",
  "model_asset_id": "6fabcd7b-7093-44b0-a99b-24035e360d87",
  "model_container_type": "catalog",
  "model_container_id": "d8d10c03-1ba0-49ff-ac58-f5df932ae29b"
}
{
  "id": "870f3666d8d6e8efdcede6fc53375cd3",
  "name": "Loan Application Model Deployment with modifications",
  "type": "online",
  "scoring_endpoint": "https://runtime.sagemaker.us-east-2.amazonaws.com/endpoints/sagemaker-scikit-learn-2023-08-11-09-00-35-455/invocations",
  "is_deleted": false,
  "description": "Model deployed in AWS SageMaker to predict t

In [48]:
deployments_all[1].set_environment_type(to_container="operate")

2024/08/05 10:47:59 INFO : Deployment successfully moved to operate environment


---
## Cleanup <a href="#clean"></a>

In [49]:
if run_cleanup_at_end:
    # remove model and model usecase
    external_model.untrack()
    facts_client.assets.remove_asset(asset_id=ai_usecase.get_info()["model_usecase_id"],container_type=ai_usecase.get_info()["container_type"],container_id=ai_usecase.get_info()["catalog_id"])
    facts_client.assets.remove_asset(asset_id=external_model.get_info()["asset_id"],container_type=external_model.get_info()["container_type"],container_id=external_model.get_info()["container_id"])

else:
    
    model_ui_url = external_model.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created wml model details in UI](" + model_ui_url + ")"))
    model_usecase_ui_url = ai_usecase.get_info(verbose=True)["url"]
    display(Markdown("[Click here to see the created model use case in UI](" + model_usecase_ui_url + ")"))

2024/08/05 10:48:03 INFO : Successfully finished unregistering WKC Model 6fabcd7b-7093-44b0-a99b-24035e360d87 from model use case.
2024/08/05 10:48:06 INFO : Successfully deleted asset id b6713b82-5730-4cff-8de5-050c13b0b80d in catalog 9df1cb81-e4d1-4b8b-9825-460f84b55dea
2024/08/05 10:48:08 INFO : Successfully deleted asset id 6fabcd7b-7093-44b0-a99b-24035e360d87 in catalog d8d10c03-1ba0-49ff-ac58-f5df932ae29b


**Created by:**  


IBM watsonx.governance - AI Factsheet Python SDK Team

---

**Copyright © 2020-2024 IBM**  
Released under the MIT License.
